# Gateway OAuth Inbound를 사용하여 AWS Lambda를 MCP화하기
## Bedrock AgentCore Gateway를 사용하여 AWS Lambda 함수를 안전한 MCP Tool로 변환

## 개요
Bedrock AgentCore Gateway는 고객이 인프라나 호스팅을 관리할 필요 없이 기존 AWS Lambda 함수를 완전 관리형 MCP 서버로 전환할 수 있는 방법을 제공합니다. Gateway는 이러한 모든 Tool에 대해 균일한 Model Context Protocol (MCP) 인터페이스를 제공합니다. Gateway는 들어오는 요청과 대상 리소스에 대한 아웃바운드 연결 모두에 대해 안전한 액세스 제어를 보장하기 위해 이중 인증 모델을 사용합니다. 프레임워크는 두 가지 주요 구성 요소로 구성됩니다: gateway 대상에 액세스하려는 사용자를 검증하고 권한을 부여하는 Inbound Auth와, 인증된 사용자를 대신하여 gateway가 백엔드 리소스에 안전하게 연결할 수 있도록 하는 Outbound Auth입니다. Gateway는 아웃바운드 권한 부여를 위해 AWS Lambda 함수 호출을 승인하는 데 IAM 역할을 사용합니다.

이 예제에서는 인바운드 권한 부여를 위한 OAuth와 아웃바운드 권한 부여를 위한 IAM 역할을 시연합니다.

![How does it work](images/lambda-iam-gateway.png)

### 튜토리얼 세부 정보


| 정보                  | 세부 사항                                                   |
|:---------------------|:----------------------------------------------------------|
| 튜토리얼 유형          | 인터랙티브                                                  |
| AgentCore 구성 요소   | AgentCore Gateway, AgentCore Identity                     |
| Agentic Framework    | Strands Agents                                            |
| Gateway Target 유형   | AWS Lambda                                                |
| Inbound Auth IdP     | Amazon Cognito                                            |
| Outbound Auth        | AWS IAM                                                   |
| LLM model            | Anthropic Claude Haiku 4.5, Amazon Nova Pro              |
| 튜토리얼 구성 요소      | AgentCore Gateway 생성 및 AgentCore Gateway 호출            |
| 튜토리얼 분야          | 범용                                                       |
| 예제 복잡도           | 쉬움                                                       |
| 사용된 SDK           | boto3                                                     |

튜토리얼의 첫 번째 부분에서는 몇 가지 AmazonCore Gateway 대상을 생성합니다

### 튜토리얼 아키텍처
이 튜토리얼에서는 AWS lambda 함수에 정의된 작업을 MCP Tool로 변환하고 Bedrock AgentCore Gateway에서 호스팅합니다.
시연 목적으로 Amazon Bedrock Model을 사용하는 Strands Agent를 사용합니다
예제에서는 get_order와 update_order라는 두 가지 Tool이 있는 매우 간단한 agent를 사용합니다.

## 사전 요구 사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Jupyter notebook (Python kernel)
* uv
* AWS credentials
* Amazon Cognito

## 들어오는 AgentCore Gateway 요청에 대한 인증 구성
AgentCore Gateway는 인바운드 및 아웃바운드 인증을 통해 안전한 연결을 제공합니다. 인바운드 인증의 경우, AgentCore Gateway는 호출 중에 전달된 OAuth 토큰을 분석하여 gateway의 Tool에 대한 액세스를 허용하거나 거부할지 결정합니다. Tool이 외부 리소스에 액세스해야 하는 경우, AgentCore Gateway는 API Key, IAM 또는 OAuth Token을 통한 아웃바운드 인증을 사용하여 외부 리소스에 대한 액세스를 허용하거나 거부할 수 있습니다.



인바운드 권한 부여 흐름 중에 agent 또는 MCP 클라이언트는 OAuth 액세스 토큰(사용자의 IdP에서 생성됨)을 추가하여 AgentCore Gateway의 MCP Tool을 호출합니다. AgentCore Gateway는 OAuth 액세스 토큰을 검증하고 인바운드 권한 부여를 수행합니다.

AgentCore Gateway에서 실행되는 Tool이 외부 리소스에 액세스해야 하는 경우, OAuth는 Gateway 대상에 대한 리소스 자격 증명 공급자를 사용하여 다운스트림 리소스의 자격 증명을 검색합니다. AgentCore Gateway는 다운스트림 API에 액세스하기 위해 호출자에게 권한 부여 자격 증명을 전달합니다.

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
# Set AWS credentials if not using Amazon SageMaker notebook
import os
# os.environ['AWS_ACCESS_KEY_ID'] = '' # Set the access key
# os.environ['AWS_SECRET_ACCESS_KEY'] = '' # Set the secret key
# AWS_REGION 환경변수가 없으면 기본값으로 us-east-1 사용
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1') # set the AWS region

In [ ]:
import os
import sys

# Get the directory of the current script
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    # Jupyter 환경에서는 __file__이 정의되지 않으므로 현재 작업 디렉토리 사용
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

# Navigate to the directory containing utils.py (one level up)
# 상위 디렉토리의 utils.py를 import하기 위한 경로 설정
utils_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add to sys.path
sys.path.insert(0, utils_dir)

# Now you can import utils
import utils

In [ ]:
#### Create a sample AWS Lambda function that you want to convert into MCP tools
# Lambda 함수 생성 (zip 파일로부터)
lambda_resp = utils.create_gateway_lambda("lambda_function_code.zip")

if lambda_resp is not None:
    if lambda_resp['exit_code'] == 0:
        print("Lambda function created with ARN: ", lambda_resp['lambda_function_arn'])
    else:
        print("Lambda function creation failed with message: ", lambda_resp['lambda_function_arn'])

In [ ]:
#### Create an IAM role for the Gateway to assume
import utils
# Gateway가 assume할 IAM 역할 생성
agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# Gateway에 대한 인바운드 권한 부여를 위한 Amazon Cognito Pool 생성

In [ ]:
# Creating Cognito User Pool 
import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
# OAuth scope 정의 (읽기/쓰기 권한)
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {user_pool_id}")

utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

# Machine-to-Machine 클라이언트 생성 (client credentials grant 방식)
client_id, client_secret  = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"Client ID: {client_id}")

# Get discovery URL  
# OIDC discovery endpoint URL 생성
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)

# 인바운드 권한 부여를 위한 Amazon Cognito Authorizer를 사용하여 Gateway 생성

In [ ]:
# CreateGateway with Cognito authorizer without CMK. Use the Cognito user pool created in the previous step
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])
# Custom JWT authorizer 설정 (Cognito 기반)
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # Client MUST match with the ClientId configured in Cognito. Example: 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='TestGWforLambda',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # The IAM Role must have permissions to create/list/get/delete Gateway 
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway with AWS Lambda target type'
)
print(create_response)
# Retrieve the GatewayID used for GatewayTarget creation
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

# AWS Lambda 대상을 생성하고 MCP Tool로 변환

In [ ]:
# Replace the AWS Lambda function ARN below
# Lambda 함수를 MCP Tool로 변환하기 위한 설정
lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": lambda_resp['lambda_function_arn'], # Replace this with your AWS Lambda function ARN
            # Tool 스키마 정의 (주문 조회/업데이트 Tool)
            "toolSchema": {
                "inlinePayload": [
                    {
                        "name": "get_order_tool",
                        "description": "tool to get the order",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "orderId": {
                                    "type": "string"
                                }
                            },
                            "required": ["orderId"]
                        }
                    },                    
                    {
                        "name": "update_order_tool",
                        "description": "tool to update the orderId",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "orderId": {
                                    "type": "string"
                                }
                            },
                            "required": ["orderId"]
                        }
                    }
                ]
            }
        }
    }
}

# Gateway IAM 역할을 사용한 자격 증명 설정 (아웃바운드 인증)
credential_config = [ 
    {
        "credentialProviderType" : "GATEWAY_IAM_ROLE"
    }
]
targetname='LambdaUsingSDK'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='Lambda Target using SDK',
    targetConfiguration=lambda_target_config,
    credentialProviderConfigurations=credential_config)

# Strands Agent에서 Bedrock AgentCore Gateway 호출

Strands agent는 Model Context Protocol (MCP) 사양을 구현하는 Bedrock AgentCore Gateway를 통해 AWS Tool과 원활하게 통합됩니다. 이 통합은 AI agent와 AWS 서비스 간의 안전하고 표준화된 통신을 가능하게 합니다.

핵심적으로 Bedrock AgentCore Gateway는 기본 MCP API인 ListTools 및 InvokeTools를 노출하는 프로토콜 준수 Gateway 역할을 합니다. 이러한 API를 통해 MCP 호환 클라이언트 또는 SDK가 안전하고 표준화된 방식으로 사용 가능한 Tool을 검색하고 상호 작용할 수 있습니다. Strands agent가 AWS 서비스에 액세스해야 할 때 이러한 MCP 표준화된 엔드포인트를 사용하여 Gateway와 통신합니다.

Gateway의 구현은 (MCP Authorization specification)[https://modelcontextprotocol.org/specification/draft/basic/authorization]을 엄격하게 준수하여 강력한 보안 및 액세스 제어를 보장합니다. 즉, Strands agent의 모든 Tool 호출은 권한 부여 단계를 거쳐 보안을 유지하면서 강력한 기능을 활성화합니다.

예를 들어, Strands agent가 MCP Tool에 액세스해야 할 때 먼저 ListTools를 호출하여 사용 가능한 Tool을 검색한 다음 InvokeTools를 사용하여 특정 작업을 실행합니다. Gateway는 필요한 모든 보안 검증, 프로토콜 변환 및 서비스 상호 작용을 처리하여 전체 프로세스를 원활하고 안전하게 만듭니다.

이러한 아키텍처 접근 방식은 MCP 사양을 구현하는 모든 클라이언트 또는 SDK가 Gateway를 통해 AWS 서비스와 상호 작용할 수 있음을 의미하며, AI agent 통합을 위한 다재다능하고 미래 지향적인 솔루션이 됩니다.

![Strands agent calling Gateway](images/strands-lambda-gateway.png)

# 인바운드 권한 부여를 위해 Amazon Cognito에서 액세스 토큰 요청

In [ ]:
import time
time.sleep(10)

In [ ]:
print("Requesting the access token from Amazon Cognito authorizer...May fail for some time till the domain name propogation completes")
# Cognito에서 OAuth 액세스 토큰 요청 (client credentials grant)
token_response = utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
token = token_response["access_token"]
print("Token response:", token)

# Bedrock AgentCore Gateway를 사용하여 AWS Lambda의 MCP Tool을 호출하는 Strands agent

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

# Gateway URL과 Bearer 토큰을 사용한 HTTP transport 생성
def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

## The IAM credentials configured in ~/.aws/credentials should have access to Bedrock model
# Amazon Nova Pro 모델 설정
yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging


# Configure the root strands logger. Change it to DEBUG if you are debugging the issue.
logging.getLogger("strands").setLevel(logging.INFO)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
    # Call the listTools 
    # Gateway에서 사용 가능한 Tool 목록 조회
    tools = client.list_tools_sync()
    # Create an Agent with the model and tools
    agent = Agent(model=yourmodel,tools=tools) ## you can replace with any model you like
    print(f"Tools loaded in the agent are {agent.tool_names}")
    # print(f"Tools configuration in the agent are {agent.tool_config}")
    # Invoke the agent with the sample prompt. This will only invoke  MCP listTools and retrieve the list of tools the LLM has access to. The below does not actually call any tool.
    agent("Hi , can you list all tools available to you")
    # Invoke the agent with sample prompt, invoke the tool and display the response
    agent("Check the order status for order id 123 and show me the exact response from the tool")
    # Call the MCP tool explicitly. The MCP Tool name and arguments must match with your AWS Lambda function or the OpenAPI/Smithy API
    # MCP Tool 직접 호출 (tool_use_id는 고유 식별자)
    result = client.call_tool_sync(
    tool_use_id="get-order-id-123-call-1", # You can replace this with unique identifier. 
    name=targetname+"___get_order_tool", # This is the tool name based on AWS Lambda target types. This will change based on the target name
    arguments={"orderId": "123"}
    )
    # Print the MCP Tool response
    print(f"Tool Call result: {result['content'][0]['text']}")


**문제: 아래 셀을 실행할 때 다음 오류가 발생하면 pydantic과 pydantic-core 버전 간의 비호환성을 나타냅니다.**

```
TypeError: model_schema() got an unexpected keyword argument 'generic_origin'
```
**해결 방법은?**

호환되는 pydantic==2.7.2와 pydantic-core 2.27.2가 모두 있는지 확인해야 합니다. 완료되면 커널을 다시 시작하세요.

# 정리

IAM 역할, IAM 정책, 자격 증명 공급자, AWS Lambda 함수, Cognito 사용자 풀, s3 버킷과 같은 추가 리소스도 생성되므로 정리의 일부로 수동으로 삭제해야 할 수 있습니다. 이는 실행하는 예제에 따라 다릅니다.

## Gateway 삭제 (선택 사항)

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)